<a href="https://colab.research.google.com/github/nedokormysh/Stepik_Ai_edu_RNN/blob/week_6_Word_RNN_Classification/AiEdu_WordRNN_Hometask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание: BiLSTM для задачи PoS Tagging

В этом ноутбуке мы будем создавать модель машинного обучения, которая генерирует результат для каждого элемента входной последовательности с использованием PyTorch и TorchText. Конкретно, мы будем подавать текст на вход, а модель будет выводить метку - часть речи (PoS) для каждого токена во входном тексте. Этот подход также может применяться для распознавания именованных сущностей (NER), где результатом для каждого токена будет указание на тип сущности, если таковая имеется.

В этом блокноте мы реализуем многослойную двунаправленную LSTM (BiLSTM) для предсказания меток частей речи с использованием набора данных Universal Dependencies English Web Treebank (UDPOS).

In [1]:
!pip install torchtext==0.6.0 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 543.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.1 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext import data
from torchtext import datasets

import spacy
import numpy as np

import time
import random

Зафиксируем случайности для воспроизводимости результатов.

In [3]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

В этом наборе данных есть два разных набора меток: метки универсальных зависимостей (UD) и метки Penn Treebank (PTB). Мы будем обучать модель только на метках UD, но загрузим метки PTB, чтобы показать, как их можно использовать вместо них.

* UD_TAGS определяет, как следует обрабатывать метки UD. В нашем словаре TEXT, который мы создадим позже, будут неизвестные токены, то есть токены, которых нет в нашем словаре. Однако у нас не будет неизвестных меток, поскольку мы имеем дело с конечным набором возможных меток. Мы будем обозначать неизвестные токены как <unk>, и затем будем их убирать, установив unk_token = None.

* PTB_TAGS выполняет то же самое, что и UD_TAGS, но обрабатывает метки PTB.

In [4]:
from torchtext.data import Field

TEXT = Field(lower = True)
UD_TAGS = Field(unk_token = None)
PTB_TAGS = Field(unk_token = None)

In [5]:
fields = (("text", TEXT), ("udtags", UD_TAGS), ("ptbtags", PTB_TAGS))

Загрузим датасет UDPOS.

In [6]:
train_data, valid_data, test_data = datasets.UDPOS.splits(fields)

downloading en-ud-v2.zip


en-ud-v2.zip: 100%|██████████| 688k/688k [00:00<00:00, 4.93MB/s]


extracting


## Задание

Посмотрите на количество объектов в датасетах `train_data, valid_data и test_data`. В ответ запишите число объектов в самом маленьком датасете.

In [7]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 12543
Number of validation examples: 2002
Number of testing examples: 2077


In [8]:
sum((1 for x in train_data))

12543

In [9]:
sum((1 for x in valid_data))

2002

In [10]:
sum((1 for x in test_data))

2077

Напечатаем пример из датасета

In [11]:
print(vars(train_data.examples[0]))

{'text': ['al', '-', 'zaman', ':', 'american', 'forces', 'killed', 'shaikh', 'abdullah', 'al', '-', 'ani', ',', 'the', 'preacher', 'at', 'the', 'mosque', 'in', 'the', 'town', 'of', 'qaim', ',', 'near', 'the', 'syrian', 'border', '.'], 'udtags': ['PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT'], 'ptbtags': ['NNP', 'HYPH', 'NNP', ':', 'JJ', 'NNS', 'VBD', 'NNP', 'NNP', 'NNP', 'HYPH', 'NNP', ',', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'NNP', ',', 'IN', 'DT', 'JJ', 'NN', '.']}


Можем отдельно посмотреть на текст и на теги

In [12]:
print(vars(train_data.examples[0])['text'])

['al', '-', 'zaman', ':', 'american', 'forces', 'killed', 'shaikh', 'abdullah', 'al', '-', 'ani', ',', 'the', 'preacher', 'at', 'the', 'mosque', 'in', 'the', 'town', 'of', 'qaim', ',', 'near', 'the', 'syrian', 'border', '.']


In [13]:
print(vars(train_data.examples[0])['udtags'])

['PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT']


In [14]:
print(vars(train_data.examples[0])['ptbtags'])

['NNP', 'HYPH', 'NNP', ':', 'JJ', 'NNS', 'VBD', 'NNP', 'NNP', 'NNP', 'HYPH', 'NNP', ',', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'NNP', ',', 'IN', 'DT', 'JJ', 'NN', '.']


Что мы сделаем дальше:

* Мы создадим словарь - отображение токенов в целые числа.

* Мы хотим, чтобы в нашем наборе данных были некоторые неизвестные токены, чтобы воссоздать, как эта модель будет использоваться в реальной жизни, поэтому мы устанавливаем `min_freq = 2`, что означает, что в словарь будут добавлены только токены, появляющиеся хотя бы дважды в обучающем наборе, и остальные будут заменены токенами `<unk>`.

* Мы также загружаем предобученные векторы GloVe длины 100 для инициализации эмбеддингов.

* `unk_init` используется для инициализации эмбеддингов токенов, которых нет в словаре предварительно обученных вложений. По умолчанию эта инициализация устанавливает эти эмбеддинги в нули, однако лучше избежать их инициализации одним и тем же значением, поэтому мы инициализируем их из нормального распределения.

* Предобученные векторы загружаем в наш словарь и будем инициализировать нашу модель этими значениями позже.

## Задание

По тренировочным данным постройте три словаря, используя `build_vocab`:

* Cловарь по текстам `TEXT` с гиперпараметрами:
  * min_freq = MIN_FREQ
  * vectors = "glove.6B.100d"
  * unk_init = torch.Tensor.normal_

* Словарь по `UD_TAGS`

* Словарь по `PTB_TAGS`

Сколько уникальных токенов в словаре, построенном по текстам?

In [15]:
MIN_FREQ = 2

TEXT.build_vocab(
    train_data,
    # max_size=vocab_size,
    min_freq = MIN_FREQ,
    vectors="glove.6B.100d",
    unk_init = torch.Tensor.normal_
)

UD_TAGS.build_vocab(train_data)
PTB_TAGS.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [02:39, 5.40MB/s]                           
100%|█████████▉| 399999/400000 [00:22<00:00, 17394.68it/s]


In [16]:
# UD_TAGS.build_vocab(
#     train_data,
#     # max_size=vocab_size,
#     min_freq = MIN_FREQ,
#     vectors="glove.6B.100d",
#     unk_init = torch.Tensor.normal_
# )

In [17]:
# PTB_TAGS.build_vocab(
#     train_data,
#     # max_size=vocab_size,
#     min_freq = MIN_FREQ,
#     vectors="glove.6B.100d",
#     unk_init = torch.Tensor.normal_
# )

In [18]:
TEXT.vocab

In [19]:
len(TEXT.vocab.itos)

8866

In [20]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in UD_TAG vocabulary: {len(UD_TAGS.vocab)}")
print(f"Unique tokens in PTB_TAG vocabulary: {len(PTB_TAGS.vocab)}")

Unique tokens in TEXT vocabulary: 8866
Unique tokens in UD_TAG vocabulary: 18
Unique tokens in PTB_TAG vocabulary: 51


## Задание

Какой самый популярный (часто встречающийся) токен в словаре, построенном по текстам?

In [21]:
TEXT.vocab.freqs.most_common(10)

[('the', 9076),
 ('.', 8640),
 (',', 7021),
 ('to', 5137),
 ('and', 5002),
 ('a', 3782),
 ('of', 3622),
 ('i', 3379),
 ('in', 3112),
 ('is', 2239)]

Посмотрим на функцию, вычисляющую процентное соотношение тегов в текстах.

In [22]:
def tag_percentage(tag_counts):

    total_count = sum([count for tag, count in tag_counts])

    tag_counts_percentages = [(tag, count, count/total_count) for tag, count in tag_counts]

    return tag_counts_percentages

## Задание

Пользуясь функцией `tag_percentage`, выведите на экран процентное соотношение каждого UD-тэга.

Какой тег встречается в текстах чаще всего (в процентах)?

In [23]:
print("Tag\t\tCount\t\tPercentage\n")

for tag, count, percent in tag_percentage(UD_TAGS.vocab.freqs.most_common()):
    print(f"{tag}\t\t{count}\t\t{percent*100:4.1f}%")

Tag		Count		Percentage

NOUN		34781		17.0%
PUNCT		23679		11.6%
VERB		23081		11.3%
PRON		18577		 9.1%
ADP		17638		 8.6%
DET		16285		 8.0%
PROPN		12946		 6.3%
ADJ		12477		 6.1%
AUX		12343		 6.0%
ADV		10548		 5.2%
CCONJ		6707		 3.3%
PART		5567		 2.7%
NUM		3999		 2.0%
SCONJ		3843		 1.9%
X		847		 0.4%
INTJ		688		 0.3%
SYM		599		 0.3%


## Задание

Используя `BucketIterator.split`, создайте объекты `train_iterator, valid_iterator, test_iterator` для итерирования по батчам.

In [24]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device
)

https://github.com/bentrevett/pytorch-pos-tagging/blob/master/1_bilstm.ipynb

## Создаем архитектуру нейронной сети

![](https://github.com/bentrevett/pytorch-pos-tagging/blob/master/assets/pos-bidirectional-lstm.png?raw=1)

Задайте нейронную сеть по аналогии с сетью из вебинара:

* Слой Embedding:
  * помимо прочего задайте `padding_idx = pad_idx`

* Затем слой LSTM с гиперпараметрами:
  * `n_layers = 1`
  * `bidirectional = True`
  * задайте `dropout`

* Затем DropOut слой

* Линейный слой, принимающий на вход `hidden_dim * 2` нейронов (так как двунаправленная сеть) и на выходе `output_dim` нейронов

В ответ на задание выберите, как выглядит первая строчка в архитектуре сети.

In [25]:
class BiLSTMPOSTagger(nn.Module):
    def __init__(self,
                 input_dim,
                 embedding_dim,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout,
                 pad_idx):

        super().__init__()

        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx = pad_idx)

        self.lstm = nn.LSTM(embedding_dim,
                            hidden_dim,
                            num_layers = n_layers,
                            bidirectional = bidirectional,
                            dropout = dropout if n_layers > 1 else 0)

        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)

        self.dropout = nn.Dropout(dropout)


    def forward(self, text):

        #pass text through embedding layer and then through dropout layer
        embedded = self.dropout(self.embedding(text))

        #pass embeddings into LSTM
        outputs, (hidden, cell) = self.lstm(embedded)

        #apply dropout and then linear layer
        predictions = self.fc(self.dropout(outputs))

        return predictions

## Обучение модели

## Задание

Запустите ячейку ниже. Если класс `BiLSTMPOSTagger` реализован корректно, ячейка отработает без ошибок. Получилось?

In [26]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 128
OUTPUT_DIM = len(UD_TAGS.vocab)
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = BiLSTMPOSTagger(INPUT_DIM,
                        EMBEDDING_DIM,
                        HIDDEN_DIM,
                        OUTPUT_DIM,
                        N_LAYERS,
                        BIDIRECTIONAL,
                        DROPOUT,
                        PAD_IDX)

Инициализируем веса сети числами из стандартного нормального распределения

In [27]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean = 0, std = 0.1)

model.apply(init_weights)

BiLSTMPOSTagger(
  (embedding): Embedding(8866, 100, padding_idx=1)
  (lstm): LSTM(100, 128, num_layers=2, dropout=0.25, bidirectional=True)
  (fc): Linear(in_features=256, out_features=18, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
)

## Задание

Напишите функцию для вычисления количества весов сети.

С помощью этой функции выведите на экран число весов нашей сети.

In [28]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,522,010 trainable parameters


## Задание

Инициализируйте embedding-слой сети предобученными GloVe-векторами.

В ответ напишите число координат в предобученных эмбеддингах.

In [29]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([8866, 100])


Инициализируем нулями pad-токены

In [30]:
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0081, -0.1063, -0.0783,  ...,  0.0307, -0.1222,  0.0106],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0287,  0.1974, -0.2067,  ...,  0.0309,  0.0399, -0.0211],
        ...,
        [ 0.0215, -0.0280,  0.0792,  ...,  0.0782,  0.0991, -0.0131],
        [ 0.1314, -0.1191, -0.0668,  ...,  0.0351, -0.1579, -0.1078],
        [ 0.0252,  0.0359, -0.1001,  ..., -0.1587,  0.0723, -0.0598]])


Зададим оптимизатор

In [31]:
optimizer = optim.Adam(model.parameters())

Зададим loss.

В случае токена `<pad>` (пустота) лосс мы не считаем, поэтому индексы таких токенов мы пропускаем (игнорируем).

In [32]:
TAG_PAD_IDX = UD_TAGS.vocab.stoi[UD_TAGS.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

Переносим модель на GPU по возможности

In [33]:
model = model.to(device)
criterion = criterion.to(device)

Функция ниже вычисляет `accuracy` для каждого батча

In [34]:
def categorical_accuracy(preds, y, tag_pad_idx):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    non_pad_elements = (y != tag_pad_idx).nonzero()
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / y[non_pad_elements].shape[0]

## Задание

Допишите цикл обучения модели.

Для каждого батча на каждой итерации:
- зануляем градиенты
- применяем модель к батчу
- делаем reshape прогнозов, так как loss нельзя вычислить для тензора размерности 3 (это уже написано)
- вычисляем loss и accuracy
- вычисляем градиенты и делаем шаг градиентного спуска

In [35]:
def train(model, iterator, optimizer, criterion, tag_pad_idx):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:

        text = batch.text
        tags = batch.udtags

        optimizer.zero_grad()

        predictions = model(text)

        predictions = predictions.view(-1, predictions.shape[-1]) # predictions - прогнозы модели
        tags = tags.view(-1) # tags - правильные ответы (метки)

        loss = criterion(predictions, tags)

        acc = categorical_accuracy(predictions, tags, tag_pad_idx)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Функцию `evaluate` для простоты мы написали.

In [36]:
def evaluate(model, iterator, criterion, tag_pad_idx):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for batch in iterator:

            text = batch.text
            tags = batch.udtags

            predictions = model(text)

            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)

            loss = criterion(predictions, tags)

            acc = categorical_accuracy(predictions, tags, tag_pad_idx)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Ниже функция, которая замеряет время обучения на каждой эпохе

In [37]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

## Задание

Обучим нашу модель. Допишите цикл по подсказкам в коде.

Какая accuracy (в процентах) получается на валидации на последней эпохе? Ответ округлите до целого числа.

In [38]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, TAG_PAD_IDX) # - примените функцию для обучения модели
    valid_loss, valid_acc = evaluate(model, test_iterator, criterion, TAG_PAD_IDX)# примените функцию для применения и оценки качества модели

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)# замерьте время выполнения эпохи, используя написанную для этого функцию

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')


    # для каждой эпохи выведите train loss, train accuracy, val loss, val accuracy, epoch time
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'train loss = {train_loss}, train accuracy = {train_acc*100}%,| val loss = {valid_loss}, val accuracy = {valid_acc*100}%')

    # print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    # print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    # print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 4s
train loss = 1.708321938709337, train accuracy = 46.18485237718845%,| val loss = 0.9638762999983395, val accuracy = 71.10648190273959%
Epoch: 02 | Epoch Time: 0m 2s
train loss = 0.5194270370566115, train accuracy = 84.02690060284674%,| val loss = 0.5766892433166504, val accuracy = 83.79984743454877%
Epoch: 03 | Epoch Time: 0m 2s
train loss = 0.30512974791380826, train accuracy = 90.70384508492995%,| val loss = 0.4826907340218039, val accuracy = 86.204835246591%
Epoch: 04 | Epoch Time: 0m 2s
train loss = 0.23240846562750486, train accuracy = 92.79214338380464%,| val loss = 0.4386892879710478, val accuracy = 86.91256607280057%
Epoch: 05 | Epoch Time: 0m 2s
train loss = 0.19992551390005617, train accuracy = 93.76049005255408%,| val loss = 0.41594840147916007, val accuracy = 88.28883381450878%
Epoch: 06 | Epoch Time: 0m 2s
train loss = 0.17648501891870888, train accuracy = 94.4484501468892%,| val loss = 0.40380264731014476, val accuracy = 88.69704849579755%
Ep

Посмотрим на качество обученной модели на тесте

In [39]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, TAG_PAD_IDX)

print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.384 |  Test Acc: 89.28%


## Инференс

Посмотрим, как модель работает на новых данных. Допишите функцию `tag_sentence` для применения обученной модели, по подсказкам ниже.

In [43]:
TEXT.vocab.stoi[]

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f8aa168d060>>,
            {'<unk>': 0,
             '<pad>': 1,
             'the': 2,
             '.': 3,
             ',': 4,
             'to': 5,
             'and': 6,
             'a': 7,
             'of': 8,
             'i': 9,
             'in': 10,
             'is': 11,
             'you': 12,
             'that': 13,
             'it': 14,
             'for': 15,
             '-': 16,
             'have': 17,
             '"': 18,
             'on': 19,
             'was': 20,
             'with': 21,
             'this': 22,
             'be': 23,
             'are': 24,
             'they': 25,
             'not': 26,
             'as': 27,
             'we': 28,
             "'s": 29,
             'my': 30,
             ')': 31,
             '(': 32,
             'do': 33,
             'will': 34,
             'he': 35,
             'at': 36,
             '?': 37,
             'b

In [47]:
def tag_sentence(model, device, sentence, text_field, tag_field):

    model.eval() # переведите модель в режим применения

    if isinstance(sentence, str):
        nlp = spacy.load('en_core_web_sm')
        tokens = [token.text for token in nlp(sentence)]
    else:
        tokens = [token for token in sentence]

    if text_field.lower:
        tokens = [t.lower() for t in tokens]

    numericalized_tokens = [text_field.vocab.stoi[t] for t in tokens]# ваш код здесь - создайте список, состоящий из переведенных в индексы токенов из словаря text_field.vocab (используйте stoi)

    unk_idx = text_field.vocab.stoi[text_field.unk_token]

    unks = [t for t, n in zip(tokens, numericalized_tokens) if n == unk_idx]

    token_tensor = torch.LongTensor(numericalized_tokens) # приведите numericalized_tokens к типу torch.LongTensor

    token_tensor = token_tensor.unsqueeze(-1).to(device)

    predictions = model(token_tensor)# ваш код здесь - примените модель к token_tenzor

    top_predictions = predictions.argmax(-1)

    predicted_tags = [tag_field.vocab.itos[t.item()] for t in top_predictions]

    return tokens, predicted_tags, unks

## Задание

Запустите две следующие ячейки. Проверим, что написанная функция работает корректно.

В ответе выберите те токены, которые были нераспознаны (их не было в обучающих данных).

In [45]:
example_index = 1

sentence = vars(train_data.examples[example_index])['text']
actual_tags = vars(train_data.examples[example_index])['udtags']

print(sentence)

['[', 'this', 'killing', 'of', 'a', 'respected', 'cleric', 'will', 'be', 'causing', 'us', 'trouble', 'for', 'years', 'to', 'come', '.', ']']


In [48]:
tokens, pred_tags, unks = tag_sentence(model,
                                       device,
                                       sentence,
                                       TEXT,
                                       UD_TAGS)

print(unks)

['respected', 'cleric']


## Задание

Проверим качество модели. Запустите ячейку ниже. В ответе укажите число неверно классифицированных токенов.

In [49]:
print("Pred. Tag\tActual Tag\tCorrect?\tToken\n")

for token, pred_tag, actual_tag in zip(tokens, pred_tags, actual_tags):
    correct = '✔' if pred_tag == actual_tag else '✘'
    print(f"{pred_tag}\t\t{actual_tag}\t\t{correct}\t\t{token}")

Pred. Tag	Actual Tag	Correct?	Token

PUNCT		PUNCT		✔		[
DET		DET		✔		this
NOUN		NOUN		✔		killing
ADP		ADP		✔		of
DET		DET		✔		a
ADJ		ADJ		✔		respected
NOUN		NOUN		✔		cleric
AUX		AUX		✔		will
AUX		AUX		✔		be
VERB		VERB		✔		causing
PRON		PRON		✔		us
NOUN		NOUN		✔		trouble
ADP		ADP		✔		for
NOUN		NOUN		✔		years
PART		PART		✔		to
VERB		VERB		✔		come
PUNCT		PUNCT		✔		.
PUNCT		PUNCT		✔		]


## Задание

Примените модель к любому предложению (на английском языке). Какая доля токенов размечена верно? Ответ напишите в комментариях.

In [53]:
sentence = 'I love this game'

tokens, pred_tags, unks = tag_sentence(model,
                                       device,
                                       sentence,
                                       TEXT,
                                       UD_TAGS)

print(unks)

[]
